
![Alt text](https://monosnap.com/image/XWHOBpayEzmvdsR3ieN34UKGIo2NQu.png) ![Alt text](https://monosnap.com/image/ZI0ytTJUYBvLXOZOQSNwHhBb5cZxx5.png)

# 使い方

最初に対象を矩形で囲み、Enter を押してください。初期切り出しが実行されます。
この処理は決定的ではなく、結果が確率的に変動します。１度目にうまく行かなくても、何度かEnterを押すとうまく行くことがあります。  

初期切り出しでうまくいっていない箇所がある場合、マウスで前景・背景の一部を指定してから Enter を押してください。

- ０ - 背景となる領域の一部を指定  
- 1 - 前景となる領域の一部を指定 
- 2 - 背景の割合の多い領域であるという指定。
- 3 - 前景の割合の多い領域であるという指定。

Enterを押さないとセグメンテーションは実行されません。

その他
- r  - リセット
 - ± - ブラシサイズを大きくする、小さくする

うまくできたら 「s」でファイル保存してください。


2018.12.6  画像のサイズを１０２４に変更しました。表示は５１２

In [ ]:
'''
 *  最初に矩形で対象を囲み、Enter を押して下さい。初期切り出しが実行されます。
 * 　０　を押すと削除指定モードになります。不要な部分をマウスで適当になぞり、Enter を押すと
 　　　指定した領域に接続する類似領域が削除されます。
 * もし必要な領域が削除されてしまったら、１を押してその領域を指定し、Enterを押します。
 * 何も指定せずに何度か Enter を押すとゴミが消えることがあります。
 
 - 0 - 背景領域を指定。
 - 1 - 前景領域を指定。
 - 2 - 背景の割合の多い領域であるという指定。
 - 3 - 前景の割合の多い領域であるという指定。
 - 9 - ９０度回転
 - R - 初期化
 - S  - 結果の保存
 - ± - ブラシサイズを大きくする、小さくする
 - Enter  - セグメンテーションの実行
'''
import numpy as np
import math
import cv2
import sys
import os
import tkinter
import tkinter.filedialog
import tkinter.messagebox

%gui tk
root=tkinter.Tk()
root.withdraw()

BLUE = [255,0,0]        # rectangle color
RED = [0,0,255]         # PR BG
GREEN = [0,255,0]       # PR FG
MAGENTA = [255,0,255]    # sure BG
BLACK = [0,0,0]
WHITE = [255,255,255]   # sure FG
MINRECTSIZE = 400 # 領域指定とそうでない操作の切り分けのための矩形面積の下限

IMAGESIZE = 512  # 強制的に画像サイズをの数字以下に縮小する。

DRAW_BG = {'color' : MAGENTA, 'val' : 0}
DRAW_FG = {'color' : WHITE, 'val' : 1}
DRAW_PR_FG = {'color' : GREEN, 'val' : 3}
DRAW_PR_BG = {'color' : RED, 'val' : 2}

# filename = "horyou2c6k.jpg" 
# filename = "yumiko.png"

def cutImage(usecolor=False):
    global orig,img,img2,orig,output,value,mask,shrinkN,filename, mouseCallBacker,gusecolor
    
    gusecolor = usecolor
    
    print( __doc__ )
    print("画像ファイルを選んで下さい")
    
    readpath = readFilePath()
    readdir,filename = os.path.split(readpath)
    
    #orig = getbase2img(cv2.imread(filename))  # 2のべき乗のサイズになるようパディング
    orig = cv2.imread(readpath)
    img = orig.copy()    
    img, halfimg = resize(img, size=1024)   # 縦横いずれも1024以下になるまで縮小
    img2 = img.copy()                        # a copy of original image
    #h0,w0 = img.shape[:2]
    #h2,w2 = halfimg.shape[:2]

    mask = np.zeros(img.shape[:2],np.uint8) # mask initialized to PR_BG
    output = np.zeros(img.shape,np.uint8)           # output image to be shown
    halfoutput = np.zeros(halfimg.shape,np.uint8) 
    
    # input and output windows
    cv2.namedWindow('output')
    cv2.namedWindow('input')
    cv2.imshow('output',halfoutput)
    cv2.imshow('input', halfimg)

    mouseCallBacker = myMouse('input')
 
    cv2.moveWindow('input',0,90)
    cv2.moveWindow('output',halfimg.shape[1],90)
    
    # print(" マウスの左ドラッグで抽出対象を囲って下さい \n")

    do_keyEventLoop()
    
    cv2.destroyAllWindows()
    cv2.waitKey(1) # jupyter notebook の場合、destroyWindowの後に waitKey しないと終了できなくなる

def do_keyEventLoop():
    global orig,img,img2,output,value,mask, rect,frame_or_mask, mouseCallBacker,filename,quitflag,framed
    framed = False
    # キーイベントループ
    while(1):
    
        k = cv2.waitKey(1) # 1milisecond 入力を受け付ける　　0にすると無限に待ってしまう

        if k == 27 or k == ord('q')  or  k ==ord('`'):         # 終了は ESC
            quitflag = True
            break
            
        elif k == ord('0'): # 背景領域の指定
            # print(" 背景領域を指定 \n")
            value = DRAW_BG
        elif k == ord('1'): # 対象の指定
            # print(" 切り出し対象領域を指定 \n")
            value = DRAW_FG
        elif k == ord('2'): # 背景かも知れない領域の指定
            # print(" 背景かも知れない領域の指定 \n")
            value = DRAW_PR_BG
        elif k == ord('3'): # 前景かもしれない領域の指定
            # print(" 前景かもしれない領域の指定 \n")
            value = DRAW_PR_FG
            
        elif k == ord('+'):
            mouseCallBacker.thicknessUp()

        elif k == ord('-'):
            mouseCallBacker.thicknessDown()
        
        elif k == ord('9'): # 90度回転
            if not mouseCallBacker.framed:
                print(" 回転します\n")
                img = img.transpose(1,0,2)[::-1,:,:]
                img2=img2.transpose(1,0,2)[::-1,:,:]
                mask=mask.transpose(1,0)[::-1,:]
                output=output.transpose(1,0,2)[::-1,:,:]
                rect= (rect[1],rect[0],rect[3],rect[2]) 
                makeupAndShowImage() 
            else:
                print(" フレーム確定後は回転できません。リセットしてください。\n")
            
        elif k == ord('s'): # 画像の保存
            bar = np.zeros((img.shape[0],5,3),np.uint8)
            res = np.hstack((img2,bar,img,bar,output))
            
            print("抽出結果を保存するパスを選んで下さい（拡張子は不要）")
            savepath = saveFilePath(filename)
            
            _ret,bw = cv2.threshold(cv2.cvtColor(output,cv2.COLOR_BGR2GRAY),1,255,cv2.THRESH_BINARY)

            savedir, ext = os.path.splitext(savepath)
            resultimg, (x0,y0,w0,h0),m = cutmargin(bw,margin=5)
            cv2.imwrite(savepath,resultimg)
            timg = np.zeros((h0+2*m,w0+2*m,3),np.uint8)
            timg[m:m+h0,m:m+w0]=img[y0:y0+h0,x0:x0+w0]
            cv2.imwrite(savedir+"Color.png",timg)
            print(savepath,"に保存しました。")
            
            cv2.imwrite('grabcut_output.png',res)
            print("抽出結果は保存先:{}に、\n, それとは別に合成画像を grabcut_output.png に結果を保存しました.\n".format(savepath+".png"))
            quitflag = False
            break
            
        elif k == ord('r'): # reset everything
            print("リセット \n")
            mouseCallBacker.init()
            img = img2 .copy()  # 画像を復元
            mask = np.zeros(img.shape[:2],np.uint8) # mask initialized to PR_BG
            output = np.zeros(img.shape,np.uint8)           # 出力画像用
            mouseCallBacker.thickness = 3
            makeupAndShowImage()
            framed = False
            
        elif k == 13 : #  Enter キー  セグメンテーションの実行
            print("セグメンテーションの実行中。新しいメッセージが表示されるまでお待ち下さい。 \n")
            if (frame_or_mask == 0):         # grabcut with rect
                bgdmodel = np.zeros((1,65),np.float64)
                fgdmodel = np.zeros((1,65),np.float64)
                mask = mask.copy()
                cv2.grabCut(img2,mask,rect,bgdmodel,fgdmodel,1,cv2.GC_INIT_WITH_RECT)
                frame_or_mask = 1
            elif frame_or_mask == 1:         # grabcut with mask
                bgdmodel = np.zeros((1,65),np.float64)
                fgdmodel = np.zeros((1,65),np.float64)
                cv2.grabCut(img2,mask,rect,bgdmodel,fgdmodel,1,cv2.GC_INIT_WITH_MASK)
            print(" 抽出がうまくいっていない場合は、手動でタッチアップしてから再度 Enter  を押して下さい。\n ０、２　背景領域の指定、１，３ 抽出対象領域の指定 \n")
            framed = True
            
        mask2 = np.where((mask==1) + (mask==3),255,0).astype('uint8')
        output = cv2.bitwise_and(img2,img2,mask=mask2)
        if gusecolor == False:
            _ret,bw = cv2.threshold(cv2.cvtColor(output,cv2.COLOR_BGR2GRAY),1,255,cv2.THRESH_BINARY)
            output =  cv2.cvtColor(bw,cv2.COLOR_GRAY2BGR)
        makeupAndShowImage()

def makeupAndShowImage():
    if mouseCallBacker.framed:
            halfimg = img[rect[1]-20:rect[1]+rect[3]+20,rect[0]-20:rect[0]+rect[2]+20]
            halfoutput = output[rect[1]-20:rect[1]+rect[3]+20,rect[0]-20:rect[0]+rect[2]+20]
    else:
            h0,w0 = img.shape[:2]
            h2,w2 = h0//2,w0//2
            halfimg = cv2.resize(img,(w2,h2))
            halfoutput = cv2.resize(output,(w2,h2)) 
    cv2.imshow('output',redmasked(halfoutput))
    cv2.imshow('input',halfimg)

class myMouse:
    def __init__(self, windowname):
        self.init()
        cv2.setMouseCallback(windowname, self.callBack, None)
        
    def init(self):
        global value,mask,framing,drawing,rect,frame_or_mask
        # setting up flags
        rect = (0,0,1,1)
        drawing = False         # 描画モードオン
        framing = False           # 選択枠設定中
        self.framed = False       # 枠設定は完了している
        frame_or_mask = 100      # flag for selecting rect or mask mode
        value =  DRAW_BG         # drawing initialized to FG
        self.thickness = 3           # ブラシサイズ
    
    def thicknessUp(self):
        self.thickness +=1
        
    def thicknessDown(self):
        if self.thickness > 0:
            self.thickness -=1
        
    def callBack(self, event, x, y, flags, param=None) :
        global img,img2,output,value,mask,framing,framed,drawing,rect,frame_or_mask
        # フレーム設定フェーズの処理
        if self.framed == False:
            if event == cv2.EVENT_LBUTTONDOWN:
                self.lx,self.ly = 2*x,2*y
                framing = True # 矩形描画モードオン
            elif event == cv2.EVENT_MOUSEMOVE:  
                if framing == True :                      
                    img = img2.copy()
                    cv2.rectangle(img,(self.lx,self.ly),(2*x,2*y),BLUE,4)
                    rect = (min(self.lx,2*x),min(self.ly,2*y),abs(self.lx-2*x),abs(self.ly-2*y))
                    
            elif event == cv2.EVENT_LBUTTONUP:
                framing = False
                tmps = abs(self.lx-2*x)*abs(self.ly-2*y)  # 指定矩形の面積
                if tmps > MINRECTSIZE:
                    self.framed = True
                    cv2.rectangle(img,(self.lx,self.ly),(2*x,2*y),BLUE,2)
                    rect = (min(self.lx,2*x),min(self.ly,2*y),abs(self.lx-2*x),abs(self.ly-2*y))
                    frame_or_mask = 0
                    print(" Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい \n")
        else: # 枠指定がすでに済んでいる場合
            imgroi = img[rect[1]-20:rect[1]+rect[3]+20,rect[0]-20:rect[0]+rect[2]+20]
            maskroi = mask[rect[1]-20:rect[1]+rect[3]+20,rect[0]-20:rect[0]+rect[2]+20]
            if drawing == True:
                if event == cv2.EVENT_MOUSEMOVE:
                    cv2.circle(imgroi,(x,y),self.thickness,value['color'],-1)
                    cv2.circle(maskroi,(x,y),self.thickness,value['val'],-1)
                elif event == cv2.EVENT_LBUTTONUP:
                    drawing = False
            else:
                if event == cv2.EVENT_LBUTTONDOWN:
                    drawing = True
        
        makeupAndShowImage()

In [ ]:
# 画像サイズの縦か横の大きい方が指定サイズになるようにリサイズし、表示用にその半サイズの画像も作る。
def resize(img, size=1024):
    maxsize = np.max(np.array(img.shape[:2]))
    height = size*img.shape[0]//maxsize
    width = size*img.shape[1]//maxsize
    output = np.zeros((height+80, width+80,3),np.uint8)
    output[40:40+height,40:40+width]=cv2.resize(img,(width,height))
    h2 = output.shape[0]//2
    w2 = output.shape[1]//2
    halfimg=cv2.resize(output,(w2,h2))
    return output,halfimg

def redmasked(img):
    ones = np.ones(img.shape[:2],np.uint8)
    zeros = np.zeros(img.shape[:2],np.uint8)
    ret,img2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV)
    b,g,r = cv2.split(img2)
    red = cv2.bitwise_and(ones,b)
    red = cv2.bitwise_and(red,g)
    red = cv2.bitwise_and(red,r)
    red = cv2.merge((zeros,zeros,red*128))
    red = cv2.bitwise_or(red,img)
    return red

# 余分なマージンをカットし関数
# 余分なマージンをカットした画像を返す関数． margin分だけ黒マージンをつける 
# 対象とする画像はシルエットが白のシルエット画像に限定
def cutmargin(img,margin=5) :
    _ret,img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY) # ２値化
    _image, contours, _hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnt = contours[np.argmax(np.array([len(c) for c in contours]))] # 最も長い輪郭
    img = cv2.drawContours(np.zeros_like(img), [cnt], -1, 255, thickness=-1) 
    x0,y0,w0,h0 = cv2.boundingRect(img)
    x,y,w,h=x0-margin,y0-margin,w0+2*margin,h0+2*margin
    resultimg = np.zeros((h,w), dtype=np.uint8)              
    resultimg[margin:h-margin,margin:w-margin]=img[y0:y0+h0,x0:x0+w0]
    return resultimg,(x0,y0,w0,h0),margin

In [ ]:
import tkinter
import tkinter.filedialog
import tkinter.messagebox
%gui tk
root=tkinter.Tk()
root.withdraw()

def readFilePath():
    fTyp=[('画像ファイルの選択',['jpg','png'])]
    readpath=tkinter.filedialog.askopenfilename(filetypes=fTyp,initialdir = './一軍')
    print("処理対象ファイル",readpath)
    return readpath

def saveFilePath(filename):                         
    savepath = tkinter.filedialog.asksaveasfilename(initialdir = "シルエット/",initialfile=filename,filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
    return savepath

while True:
    cutImage(usecolor=False)
    if quitflag:
        break